# Actividad Colaborativa 1

## 1. Nuevo dataset
Sacado de: https://www.kaggle.com/kazanova/sentiment140#

In [1]:
import re
import csv
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import collections
from nltk.metrics.scores import (precision, recall)
import pickle

In [2]:
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# Esta funcion elimina links y nombres de usuarios
def remove_linksT(text):
    text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', text)
    text = re.sub('\@\w+', ' ', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [4]:
# Este objeto es nuestro clasificador
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [5]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [6]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/NLP

/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Copia de machine-learning_speech-recognition_7601-291468-0006.wav'
 modelos
 ModelosNuevos.ipynb
 noemoticon.csv
 private.py


In [7]:
# Bajamos base de datos
documents = []
all_words = []
instancias = 5000
LB, UB = instancias, 1600000 - instancias
# Guardaremos verbos, advervios y adjetivos
allowed_word_types = ["J","R","V","N"]
with open('/mydrive/NLP/noemoticon.csv', 'r',encoding='latin-1') as file:
    csv_file = csv.reader(file)
    count=0
    for row in csv_file:
        if count < LB or count >= UB:
          # normalizamos el tweet
          text = remove_linksT(row[5].lower())
          # obtenemos todas las palabras
          words = word_tokenize(text)
          pos = nltk.pos_tag(words)
          # Guardamos el tweet y su clasificacion
          row[0] = 'neg' if row[0] == '0' else 'pos'
          documents.append((text, row[0]))
          # Guardamos las palabras
          for w in pos:
            if w[1][0] in allowed_word_types:
              all_words.append(w[0])
        count += 1 

In [8]:
countp = 0
countn = 0
countne = 0
for i in documents:
    if i[1] == 'pos':
        countp += 1
    if i[1] == 'neg':
        countn += 1
    if i[1] == '2':
        countne +=1
print(f'tenemos {countp} tweets positivos')
print(f'tenemos {countn} tweets negativos')
print(f'tenemos {countne} tweets neutrales')
print(f'tenemos {len(documents)} tweets')

tenemos 5000 tweets positivos
tenemos 5000 tweets negativos
tenemos 0 tweets neutrales
tenemos 10000 tweets


In [9]:
# vemos que tanto se repite cada palabra
all_words = nltk.FreqDist(all_words)
print(f'Tenemos {len(list(all_words.keys()))} palabras')

Tenemos 12894 palabras


In [10]:
# Tomamos las primeras
word_features = list(all_words.keys())[:600]

In [11]:
# Creamos nuestro featureset (que palabras estan en cada tweet y cual es su categoria)
featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)

In [12]:
train = int(instancias*2*0.8)
print(train)

8000


In [13]:
# dividimos en training y test set
testing_set = featuresets[train:]
training_set = featuresets[:train]

## 5. integrar tres RNN

In [14]:
import gensim
from nltk.tokenize.treebank import TreebankWordDetokenizer

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))
        
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [16]:
#def tokenize(data):
  


In [17]:
temp = []
#Splitting pd.Series to list
data_to_list = [x[0] for x in documents]
labels = [x[1] for x in documents]

#data_to_list = train['selected_text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(remove_linksT(data_to_list[i]))
data_words = list(sent_to_words(temp))
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
num_words = len(list(all_words.keys()))
max_len = 200
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)

In [19]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(tweets,labels,test_size=0.20,random_state=22)


enc = OneHotEncoder(handle_unknown='ignore')

Y_train = [[x] for x in Y_train]
Y_test = [[x] for x in Y_test]

Y_train = enc.fit_transform(Y_train).toarray()
Y_test = enc.fit_transform(Y_test).toarray()

In [20]:
file = open('/mydrive/NLP/modelos/X_train.pickle', 'wb')
pickle.dump(X_train, file)
file.close()

file = open('/mydrive/NLP/modelos/X_test.pickle', 'wb')
pickle.dump(X_test, file)
file.close()

file = open('/mydrive/NLP/modelos/Y_train.pickle', 'wb')
pickle.dump(Y_train, file)
file.close()

file = open('/mydrive/NLP/modelos/Y_test.pickle', 'wb')
pickle.dump(Y_test, file)
file.close()

In [21]:
from keras.models import Sequential
from keras import layers


In [22]:
model = Sequential()
model.add(layers.Embedding(num_words, 50)) #The embedding layer
model.add(layers.LSTM(25,dropout=0.5)) #Our LSTM layer
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(2,activation='softmax'))

In [23]:

model0 = Sequential()
model0.add(layers.Embedding(num_words, 60))
model0.add(layers.SimpleRNN(25,return_sequences=True))
model0.add(layers.SimpleRNN(25))
model0.add(layers.Dense(2,activation='softmax'))


In [24]:
model2 = Sequential()
model2.add(layers.Embedding(num_words, 100, input_length=200))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.5)))
model2.add(layers.Dense(2,activation='softmax'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          644700    
_________________________________________________________________
lstm (LSTM)                  (None, 25)                7600      
_________________________________________________________________
dense (Dense)                (None, 25)                650       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 52        
Total params: 653,002
Trainable params: 653,002
Non-trainable params: 0
_________________________________________________________________


In [26]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 100)          1289400   
_________________________________________________________________
bidirectional (Bidirectional (None, 40)                19360     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 82        
Total params: 1,308,842
Trainable params: 1,308,842
Non-trainable params: 0
_________________________________________________________________


In [27]:
model0.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 60)          773640    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 25)          2150      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 25)                1275      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 52        
Total params: 777,117
Trainable params: 777,117
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
model.compile(
    optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [29]:

# Compile the model 0
model0.compile(
    optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
# Compile the model 2
model2.compile(
    optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
model.fit(X_train, Y_train, epochs=20,validation_data=(X_test, Y_test),verbose=1)

Epoch 1/20
250/250 [==============================] - 37s 16ms/step - loss: 0.6763 - accuracy: 0.5749 - val_loss: 0.5555 - val_accuracy: 0.7210
Epoch 2/20
250/250 [==============================] - 3s 12ms/step - loss: 0.5199 - accuracy: 0.7460 - val_loss: 0.5142 - val_accuracy: 0.7445
Epoch 3/20
250/250 [==============================] - 3s 12ms/step - loss: 0.4414 - accuracy: 0.8025 - val_loss: 0.5033 - val_accuracy: 0.7590
Epoch 4/20
250/250 [==============================] - 3s 12ms/step - loss: 0.3989 - accuracy: 0.8238 - val_loss: 0.4984 - val_accuracy: 0.7665
Epoch 5/20
250/250 [==============================] - 3s 12ms/step - loss: 0.3893 - accuracy: 0.8341 - val_loss: 0.5130 - val_accuracy: 0.7590
Epoch 6/20
250/250 [==============================] - 3s 12ms/step - loss: 0.3531 - accuracy: 0.8465 - val_loss: 0.5072 - val_accuracy: 0.7550
Epoch 7/20
250/250 [==============================] - 3s 12ms/step - loss: 0.3343 - accuracy: 0.8546 - val_loss: 0.5069 - val_accuracy: 0.764

In [33]:
model0.fit(X_train, Y_train, epochs=20,validation_data=(X_test, Y_test),verbose=1)

Epoch 1/20
250/250 [==============================] - 48s 188ms/step - loss: 0.6865 - accuracy: 0.5387 - val_loss: 0.5772 - val_accuracy: 0.7060
Epoch 2/20
250/250 [==============================] - 47s 187ms/step - loss: 0.4482 - accuracy: 0.7898 - val_loss: 0.5895 - val_accuracy: 0.7105
Epoch 3/20
250/250 [==============================] - 47s 188ms/step - loss: 0.2419 - accuracy: 0.9105 - val_loss: 0.7254 - val_accuracy: 0.7035
Epoch 4/20
250/250 [==============================] - 47s 187ms/step - loss: 0.1267 - accuracy: 0.9523 - val_loss: 0.8895 - val_accuracy: 0.6860
Epoch 5/20
250/250 [==============================] - 46s 185ms/step - loss: 0.0780 - accuracy: 0.9725 - val_loss: 1.0609 - val_accuracy: 0.6685
Epoch 6/20
250/250 [==============================] - 47s 187ms/step - loss: 0.0439 - accuracy: 0.9841 - val_loss: 1.1765 - val_accuracy: 0.6650
Epoch 7/20
250/250 [==============================] - 46s 185ms/step - loss: 0.0317 - accuracy: 0.9886 - val_loss: 1.5268 - val_ac

In [34]:
model2.fit(X_train, Y_train, epochs=15,validation_data=(X_test, Y_test),verbose=1)

Epoch 1/15
250/250 [==============================] - 9s 24ms/step - loss: 0.6755 - accuracy: 0.5615 - val_loss: 0.5520 - val_accuracy: 0.7255
Epoch 2/15
250/250 [==============================] - 5s 21ms/step - loss: 0.5116 - accuracy: 0.7623 - val_loss: 0.5075 - val_accuracy: 0.7550
Epoch 3/15
250/250 [==============================] - 5s 21ms/step - loss: 0.4290 - accuracy: 0.8098 - val_loss: 0.4979 - val_accuracy: 0.7615
Epoch 4/15
250/250 [==============================] - 5s 21ms/step - loss: 0.3997 - accuracy: 0.8202 - val_loss: 0.5043 - val_accuracy: 0.7640
Epoch 5/15
250/250 [==============================] - 5s 21ms/step - loss: 0.3610 - accuracy: 0.8460 - val_loss: 0.5016 - val_accuracy: 0.7625
Epoch 6/15
250/250 [==============================] - 5s 21ms/step - loss: 0.3547 - accuracy: 0.8514 - val_loss: 0.5209 - val_accuracy: 0.7625
Epoch 7/15
250/250 [==============================] - 5s 20ms/step - loss: 0.3237 - accuracy: 0.8666 - val_loss: 0.5222 - val_accuracy: 0.7550

In [35]:
model.save('/mydrive/NLP/modelos/model.h5')
model0.save('/mydrive/NLP/modelos/model0.h5')
model2.save('/mydrive/NLP/modelos/model2.h5')

## Extra 1: Analisis sentimientos con BERT

In [36]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.6MB/s 
     |████████████████████████████████| 3.3MB 51.4MB/s 
     |████████████████████████████████| 901kB 53.0MB/s 


In [37]:
import tensorflow as tf
import pandas as pd

**Carga modelos preentrenados**

In [38]:
# carga modelo tokenizador preentrenado
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

modelBERT = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# imprime resumen del modelo
modelBERT.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


**Descarga de datos**

In [39]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

84131840/84125825 [==============================] - 2s 0us/step


In [40]:
# Importa librerias
import os
import shutil
# Se crea un directorio ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# Se crea un subdirectorio ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Se quitan los datos no etiquetados
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

**Se dividen los datos en test y train**

In [41]:
# Se crea un conunto de entrenamiento y de validacion 
# del directorio "aclImdb/train" con una division 80/20.
train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=3000, validation_split=0.2, 
    subset='training', seed=123)
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=3000, validation_split=0.2, 
    subset='validation', seed=123)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


**Visualizacion de datos**

In [42]:
# Para entrenamiento
for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
train.head()

,DATA_COLUMN,LABEL_COLUMN
0,Canadian director Vincenzo Natali took the art...,1
1,I gave this film 10 not because it is a superb...,1
2,I admit to being somewhat jaded about the movi...,1
3,"For a long time, 'The Menagerie' was my favori...",1
4,A truly frightening film. Feels as if it were ...,0


In [43]:
# Para test
for j in test.take(1):
  test_feat = j[0].numpy()
  test_lab = j[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test.head()

,DATA_COLUMN,LABEL_COLUMN
0,I can't believe that so much talent can be was...,0
1,This movie blows - let's get that straight rig...,0
2,"The saddest thing about this ""tribute"" is that...",0
3,I'm only rating this film as a 3 out of pity b...,0
4,Something surprised me about this movie - it w...,1


**Crear secuencias de inputs**

Tenemos dos funciones

1. Tomara los datos de test y entrenemiento y convierte cada linea a un objeto InputExample

2. Tokenizara los objetos InputExample y transformara los datos a datos que puedan ser procesados por el modelo

In [44]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> Retendra las caracteristicas para posteriormente ser convertidas

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # se trunca si len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            padding='max_length', 
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [45]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

**Ajuste de hiperparametros**

Se utilizara un optimizador Adam y Entropia cruzada categorizada como la funcion de pérdida

In [46]:
modelBERT.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [47]:
modelBERT.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [51]:
# Guarda los pesos
modelBERT.save_weights('/mydrive/NLP/modelos/bert_weights.h5')

**Hacer predicciones**

In [57]:
pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                  'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie']

tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = modelBERT(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good : 
 Positive
One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie : 
 Negative


In [75]:
test_data_column = test['DATA_COLUMN'].tolist()
test_data_column = test_data_column[0:100]

tf_batch = tokenizer(test_data_column, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = modelBERT(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
print(label)

[0 0 0 0 1 0 0 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0
 1 1 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1
 0 1 1 1 0 1 1 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 0]


In [78]:
m = tf.keras.metrics.Recall()
m.update_state(test_lab[0:100], label)
m.result().numpy()

0.877551